In [2]:
from parsita import *
from parsita.util import constant

# JSON definition according to https://tools.ietf.org/html/rfc7159


class JsonStringParsers(TextParsers, whitespace=None):
    quote = lit(r'\"') > constant('"')
    reverse_solidus = lit(r'\\') > constant('\\')
    solidus = lit(r'\/') > constant('/')
    backspace = lit(r'\b') > constant('\b')
    form_feed = lit(r'\f') > constant('\f')
    line_feed = lit(r'\n') > constant('\n')
    carriage_return = lit(r'\r') > constant('\r')
    tab = lit(r'\t') > constant('\t')
    uni = reg(r'\\u([0-9a-fA-F]{4})') > (lambda x: chr(int(x.group(1), 16)))

    escaped = (quote | reverse_solidus | solidus | backspace | form_feed
               | line_feed | carriage_return | tab | uni)
    unescaped = reg(r'[\u0020-\u0021\u0023-\u005B\u005D-\U0010FFFF]+')

    string = '"' >> rep(escaped | unescaped) << '"' > ''.join


class JsonParsers(TextParsers, whitespace=r'[ \t\n\r]*'):
    number = reg(r'-?(0|[1-9][0-9]*)(\.[0-9]+)?([eE][-+]?[0-9]+)?') > float

    false = lit('false') > constant(False)
    true = lit('true') > constant(True)
    null = lit('null') > constant(None)

    string = JsonStringParsers.string

    array = '[' >> repsep(value, ',') << ']'

    entry = string << ':' & value
    obj = '{' >> repsep(entry, ',') << '}' > dict

    value = number | false | true | null | string | array | obj


if __name__ == '__main__':
    strings = [
        '"name"',
        '-12.40e2',
        '[false, true, null]',
        '{"__class__" : "Point", "x" : 2.3, "y" : -1.6}',
        '{"__class__" : "Rectangle", "location" : {"x":-1.3,"y":-4.5}, "height" : 2.0, "width" : 4.0}',
        '{"text" : ""}',
    ]

    for string in strings:
        print('source: {}\nvalue: {}'.format(string, JsonParsers.value.parse(string)))

source: "name"
value: Success('name')
source: -12.40e2
value: Success(-1240.0)
source: [false, true, null]
value: Success([False, True, None])
source: {"__class__" : "Point", "x" : 2.3, "y" : -1.6}
value: Success({'__class__': 'Point', 'x': 2.3, 'y': -1.6})
source: {"__class__" : "Rectangle", "location" : {"x":-1.3,"y":-4.5}, "height" : 2.0, "width" : 4.0}
value: Success({'__class__': 'Rectangle', 'location': {'x': -1.3, 'y': -4.5}, 'height': 2.0, 'width': 4.0})
source: {"text" : ""}
value: Success({'text': ''})


In [6]:
from parsita import *


class ExpressionParsers(TextParsers):
    number = reg(r'[+-]?\d+(\.\d+)?(e[+-]?\d+)?') > float

    base = '(' >> expr << ')' | number

    factor = base & opt('^' >> base) > (lambda x: x[0] ** x[1][0] if x[1] else x[0])

    def make_term(args):
        factor1, factors = args
        result = factor1
        for op, factor in factors:
            if op == '*':
                result = result * factor
            else:
                result = result / factor
        return result
    term = factor & rep(lit('*', '/') & factor) > make_term

    def make_expr(args):
        term1, terms = args
        result = term1
        for op, term2 in terms:
            if op == '+':
                result = result + term2
            else:
                result = result - term2
        return result
    expr = term & rep(lit('+', '-') & term) > make_expr


if __name__ == '__main__':
    expressions = ['123', '2 ^ 3', '1 + 1', '1 - 2 + 3 - 4', '3 - 4 * 2 + 10', '14 / (3.1 + 3.9)']

    for expression in expressions:
        print('{} = {}'.format(expression, ExpressionParsers.expr.parse(expression).value))

123 = 123.0
2 ^ 3 = 8.0
1 + 1 = 2.0
1 - 2 + 3 - 4 = -2.0
3 - 4 * 2 + 10 = 5.0
14 / (3.1 + 3.9) = 2.0
